In [48]:
from lxml import html
import requests
from datetime import datetime
import pandas as pd
import locale

#Русская локаль для чтения месяца
locale.setlocale(locale.LC_ALL, "")

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}

#Парсинг mail.ru
link_mail = 'https://www.mail.ru'
request = requests.get(link_mail, headers=headers)
root = html.fromstring(request.text)
links = root.xpath('//div[contains(@class, "news-item i-fade-white")]/div/a[last()]/@href')
texts = root.xpath('//div[contains(@class, "news-item i-fade-white")]/div/a[last()]/text()')
main_news_link = root.xpath('//div[contains(@class, "news-item_main")]/a/@href')[0]
main_news_text = root.xpath('//div[contains(@class, "news-item_main")]/a/div/div/h3/text()')[0]

mail_news_dict = {}
links.append(main_news_link)
texts.append(main_news_text)
mail_news_dict['Ссылки'] = links
mail_news_dict['Тексты'] = texts
df_mail = pd.DataFrame(mail_news_dict)
df_mail['Дата'] = datetime.date(datetime.today())
df_mail['Источник'] = 'mail.ru'

#Парсинг lenta.ru
link_lenta = 'https://lenta.ru'
request_lenta = requests.get(link_lenta, headers=headers)
root_lenta = html.fromstring(request_lenta.text)
links_lenta_temp = root_lenta.xpath('//section/div[contains(@class, "span4")]/div[contains(@class, "item")]/a/@href')
links_lenta = []
for link in links_lenta_temp:
    links_lenta.append(link_lenta+link)
texts_lenta = root_lenta.xpath('//section/div[contains(@class, "span4")]/div[contains(@class, "item")]/a/text()')
dates_temp = root_lenta.xpath('//section/div[contains(@class, "span4")]/div[contains(@class, "item")]/a/time/@title')
lenta_main_data = root_lenta.xpath('//section/div[contains(@class, "span4")]/div[contains(@class, "item")]/h2/a/time/@title')
dates_temp.insert(0, lenta_main_data[0])
dates = []
for date in dates_temp:
    date = date.replace('я ', 'ь ') #Перевод месяца в именительный падеж
    date = date.replace('а ', ' ') #Отдельно для августа
    dates.append(datetime.date(datetime.strptime(date, "%d %B %Y")))
lenta_main_text = root_lenta.xpath('//section/div[contains(@class, "span4")]/div[contains(@class, "item")]/h2/a/text()')
texts_lenta.insert(0, lenta_main_text[0])
    
lenta_news_dict = {}
lenta_news_dict['Ссылки'] = links_lenta
lenta_news_dict['Тексты'] = texts_lenta
lenta_news_dict['Дата'] = dates
df_lenta = pd.DataFrame(lenta_news_dict)
df_lenta['Источник'] = 'lenta.ru'

#Объединим
df_all_news = pd.concat([df_mail, df_lenta], ignore_index=True)